<a href="https://colab.research.google.com/github/Julietaortizbustos/PROYECTOCHATBOT/blob/main/Chatbotfarmacogenomica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from google.colab import files
uploaded = files.upload()
df=pd.read_csv('DatasetAEMPS.csv', encoding="latin1", sep=';')

Saving DatasetAEMPS.csv to DatasetAEMPS.csv


In [ ]:
df.head()


,Columna1,Principio activo,Código ATC,Biomarcador,Biomarcador/subtipo/variante con implicación clínica,Clase de Biomarcador,Descripción de la información de la relación principio activo-biomarcador:,Incluído en la cartera de servicio nacional
0,NaN,amitriptilina,N06AA09,CYP2C19,Metabolizadores lentos,Germinal,Concentraciones plasmáticas más altas de amitr...,si
1,NaN,amitriptilina,N06AA09,CYP2D6,Metabolizadores lentos,Germinal,Concentraciones plasmáticas más altas de amitr...,si
2,NaN,aripiprazol,N05AX12,CYP2D6,Metabolizadores lentos,Germinal,Concentraciones sistémicas más altas y riesgo ...,si
3,NaN,atomoxetina,N06BA09,CYP2D6,Metabolizadores lentos,Germinal,Deberá considerarse una dosis de inicio menor ...,propuesta de inclusión
4,NaN,carbamazepina,N03AF01,HLA-A,HLA-A*31:01 alelo positivo,Germinal,La presencia del alelo HLA-A *31:01 puede aum...,si


In [ ]:
clas_df = df.copy()
clas_df.drop('Columna1', axis=1, inplace=True)
display(clas_df.head())
print(clas_df.info())
print(clas_df.describe())

,Principio activo,Código ATC,Biomarcador,Biomarcador/subtipo/variante con implicación clínica,Clase de Biomarcador,Descripción de la información de la relación principio activo-biomarcador:,Incluído en la cartera de servicio nacional
0,amitriptilina,N06AA09,CYP2C19,Metabolizadores lentos,Germinal,Concentraciones plasmáticas más altas de amitr...,si
1,amitriptilina,N06AA09,CYP2D6,Metabolizadores lentos,Germinal,Concentraciones plasmáticas más altas de amitr...,si
2,aripiprazol,N05AX12,CYP2D6,Metabolizadores lentos,Germinal,Concentraciones sistémicas más altas y riesgo ...,si
3,atomoxetina,N06BA09,CYP2D6,Metabolizadores lentos,Germinal,Deberá considerarse una dosis de inicio menor ...,propuesta de inclusión
4,carbamazepina,N03AF01,HLA-A,HLA-A*31:01 alelo positivo,Germinal,La presencia del alelo HLA-A *31:01 puede aum...,si


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 7 columns):
 #   Column                                                                      Non-Null Count  Dtype 
---  ------                                                                      --------------  ----- 
 0   Principio activo                                                            26 non-null     object
 1   Código ATC                                                                  26 non-null     object
 2   Biomarcador                                                                 26 non-null     object
 3   Biomarcador/subtipo/variante con implicación clínica                        15 non-null     object
 4   Clase de Biomarcador                                                        26 non-null     object
 5   Descripción de la información de la relación principio activo-biomarcador:  26 non-null     object
 6   Incluído en la cartera de servicio nacional                  

In [ ]:
mkdir -p data/raw/pdfs

In [ ]:
from pathlib import Path

PDF_DIR = Path("data/raw/pdfs")
pdf_files = sorted(PDF_DIR.glob("*.pdf"))

print(f"📄 PDFs detectados: {len(pdf_files)}")
for pdf in pdf_files:
    print(" -", pdf.name)

📄 PDFs detectados: 27
 - ATOMOXETINA.pdf
 - Actualizacioncpicatd-supplement.pdf
 - Actualizacioncpicatd.pdf
 - CPICATC.pdf
 - CPICATD.pdf
 - CPIC_HLA_CBZ_OXC.pdf
 - DPWG_May_2021.pdf
 - Ficha tecnica amitriptilina.pdf
 - Ficha tecnica aripiprazol.pdf
 - Ficha tecnica atomoxetina.pdf
 - Ficha tecnica carbamazepina.pdf
 - Ficha tecnica citalopram.pdf
 - Ficha tecnica clomipramina.pdf
 - Ficha tecnica doxepina.pdf
 - Ficha tecnica escitalopram.pdf
 - Ficha tecnica fluvoxamina.pdf
 - Ficha tecnica haloperidol.pdf
 - Ficha tecnica imipramina.pdf
 - Ficha tecnica lamotrigina.pdf
 - Ficha tecnica oxcarbazepina.pdf
 - Ficha tecnica paroxetina.pdf
 - Ficha tecnica quetiapina.pdf
 - Ficha tecnica risperidona.pdf
 - Ficha tecnica sertralina.pdf
 - Ficha tecnica venlafaxina.pdf
 - Ficha tecnica vortioxetina.pdf
 - Ficha tecnica zulopentixol.pdf


In [ ]:
import json
import os

# === 1. Crear carpeta de salida ===
os.makedirs("data/processed", exist_ok=True)

# === 2. Normalizar nombres de columnas ===
clas_df.columns = [c.strip().lower() for c in clas_df.columns]
print("Columnas del dataset:", clas_df.columns.tolist())

# === 3. Generar fragmentos JSONL ===
records = []
for i, row in clas_df.iterrows():
    drug = str(row.get("medicamento", "")).strip()
    gene = str(row.get("gen", "")).strip()
    evidence = str(row.get("nivel de evidencia", "")).strip() or str(row.get("evidencia", "")).strip()
    source = str(row.get("fuente", "")).strip()
    descripcion = str(row.get("descripcion", "")).strip() or str(row.get("comentario", "")).strip()

    # Construir texto descriptivo
    text_parts = []
    if drug:
        text_parts.append(f"Medicamento: {drug}")
    if gene:
        text_parts.append(f"Gen asociado: {gene}")
    if evidence:
        text_parts.append(f"Nivel de evidencia: {evidence}")
    if descripcion:
        text_parts.append(f"Descripción: {descripcion}")

    text = " - ".join(text_parts)

    fragment = {
        "id": f"aemps_{i+1:04d}",
        "text": text,
        "drug": drug,
        "gene": gene,
        "evidence": evidence,
        "source": source or "https://www.aemps.gob.es/medicamentos-de-uso-humano/base-de-datos-de-biomarcadores-farmacogenomicos/"
    }
    records.append(fragment)

# === 4. Guardar en JSONL ===
output_file = "data/processed/fragments.jsonl"
with open(output_file, "w", encoding="utf-8") as f:
    for r in records:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"✅ Archivo generado: {output_file}")
print(f"Total de fragmentos creados: {len(records)}")


Columnas del dataset: ['principio activo', 'código atc', 'biomarcador', 'biomarcador/subtipo/variante con implicación clínica', 'clase de biomarcador', 'descripción de la información de la relación principio activo-biomarcador:', 'incluído en la cartera de servicio nacional']
✅ Archivo generado: data/processed/fragments.jsonl
Total de fragmentos creados: 26


In [ ]:
!python -m venv venv
!source venv/Scripts/activate

!pip install -U pip
!pip install pandas sentence-transformers faiss-cpu pdfplumber tiktoken nltk tqdm

Error: Command '['/content/venv/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: line 1: venv/Scripts/activate: No such file or directory
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 103.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 73.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pdfplumber]


In [ ]:
import os, json, re
import pdfplumber
from tqdm import tqdm
from pathlib import Path
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# --- parámetros de chunking ---
MAX_CHARS = 1000     # tamaño objetivo por chunk (aprox. 400-700 tokens)
OVERLAP = 200        # solapamiento en caracteres


In [ ]:
PDF_DIR = "data/raw/pdfs"
INPUT_FRAGMENTS = "data/processed/fragments.jsonl"   # tus fragments AEMPS ya existentes
OUTPUT_FRAGMENTS = "data/processed/fragments_all.jsonl"

In [ ]:
def extract_text_from_pdf(path):
    text = []
    with pdfplumber.open(path) as pdf:
        for p in pdf.pages:
            t = p.extract_text()
            if t:
                text.append(t)
    return "\n".join(text)

In [ ]:
def chunk_text(text, max_chars=MAX_CHARS, overlap=OVERLAP):
    # Primero dividimos en frases, para no cortar oraciones
    sents = sent_tokenize(text)
    chunks = []
    cur = ""
    for sent in sents:
        if len(cur) + len(sent) + 1 <= max_chars:
            cur = cur + " " + sent if cur else sent
        else:
            if cur:
                chunks.append(cur.strip())
            # start new chunk, include overlap from the end of cur
            cur = sent
    if cur:
        chunks.append(cur.strip())
    # opcional: enforce overlap by merging neighbor chunks if too small
    return chunks

In [ ]:
# --- cargar fragments existentes (si existen) ---
existing = []
if os.path.exists(INPUT_FRAGMENTS):
    with open(INPUT_FRAGMENTS, "r", encoding="utf-8") as f:
        for line in f:
            existing.append(json.loads(line))


In [ ]:
# start ids from existing count
start_i = len(existing)


In [ ]:
# --- procesar PDFs ---
new_fragments = []
pdf_files = sorted(Path(PDF_DIR).glob("*.pdf"))
for pdf_path in tqdm(pdf_files, desc="Procesando PDFs"):
    text = extract_text_from_pdf(str(pdf_path))
    if not text.strip():
        continue
    # clean basic artifacts
    text = re.sub(r'\s+', ' ', text).strip()
    chunks = chunk_text(text)
    for j, chunk in enumerate(chunks):
        frag = {
            "id": f"pdf_{pdf_path.stem}_{j+1}",
            "text": chunk,
            "drug": "",   # opcional: podrías intentar extraer fármaco por palabra clave después
            "gene": "",
            "evidence": "",
            "source": str(pdf_path)
        }
        new_fragments.append(frag)

# --- combinar y guardar todo en fragments_all.jsonl ---
all_fragments = existing + new_fragments
os.makedirs(os.path.dirname(OUTPUT_FRAGMENTS), exist_ok=True)
with open(OUTPUT_FRAGMENTS, "w", encoding="utf-8") as out:
    for r in all_fragments:
        out.write(json.dumps(r, ensure_ascii=False) + "\n")

print(f"Guardado {len(new_fragments)} nuevos fragmentos desde PDFs.")
print(f"Total fragments guardados: {len(all_fragments)} -> {OUTPUT_FRAGMENTS}")

Procesando PDFs: 100%|██████████| 27/27 [03:20<00:00,  7.42s/it]

Guardado 2734 nuevos fragmentos desde PDFs.
Total fragments guardados: 2760 -> data/processed/fragments_all.jsonl


In [ ]:
import os
import pandas as pd

# Creamos carpeta donde guardaremos las listas
os.makedirs("src", exist_ok=True)

# --- 1. Revisar columnas disponibles ---
print("Columnas del clas_df:", clas_df.columns.tolist())

# Si tus columnas tienen nombres distintos (por ejemplo 'Medicamento (principio activo)' o 'Gen asociado'),
# cámbialos en estas líneas según aparezcan.
drug_col = 'principio activo'
gene_col = 'biomarcador'

print(f"Columna de principio activo detectada: {drug_col}")
print(f"Columna de biomarcador: {gene_col}")

# --- 2. Extraer valores únicos ---
drugs = (
    clas_df[drug_col]
    .dropna()
    .astype(str)
    .str.strip()
    .str.lower()
    .replace(r"[^a-záéíóúñü0-9\s\-]", "", regex=True)
    .unique()
    .tolist()
)

genes = (
    clas_df[gene_col]
    .dropna()
    .astype(str)
    .str.strip()
    .str.upper()
    .replace(r"[^A-Z0-9\*\-]", "", regex=True)
    .unique()
    .tolist()
)

# --- 3. Guardar en archivos .txt ---
drug_path = "src/drug_list.txt"
gene_path = "src/gene_list.txt"

with open(drug_path, "w", encoding="utf-8") as f:
    for d in sorted(drugs):
        f.write(d + "\n")

with open(gene_path, "w", encoding="utf-8") as f:
    for g in sorted(genes):
        f.write(g + "\n")

print(f"✅ Guardados {len(drugs)} principio activo en {drug_path}")
print(f"✅ Guardados {len(genes)} biomarcador en {gene_path}")

Columnas del clas_df: ['principio activo', 'código atc', 'biomarcador', 'biomarcador/subtipo/variante con implicación clínica', 'clase de biomarcador', 'descripción de la información de la relación principio activo-biomarcador:', 'incluído en la cartera de servicio nacional']
Columna de principio activo detectada: principio activo
Columna de biomarcador: biomarcador
✅ Guardados 21 principio activo en src/drug_list.txt
✅ Guardados 6 biomarcador en src/gene_list.txt


In [ ]:
drug_list = [d.strip().lower() for d in open("src/drug_list.txt", encoding="utf-8").read().splitlines() if d.strip()]
gene_list = [g.strip().upper() for g in open("src/gene_list.txt", encoding="utf-8").read().splitlines() if g.strip()]


In [ ]:
# src/build_index.py  (parte 2: embeddings + FAISS)
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

FRAGMENTS_FILE = "data/processed/fragments_all.jsonl"
INDEX_DIR = "indexes"
INDEX_FILE = f"{INDEX_DIR}/faiss_index.bin"
META_FILE = f"{INDEX_DIR}/metadata.jsonl"
EMBED_MODEL = "all-mpnet-base-v2"   # buen equilibrio multilingual/precisión

os.makedirs(INDEX_DIR, exist_ok=True)

# 1) cargar fragments_all
fragments = []
with open(FRAGMENTS_FILE, "r", encoding="utf-8") as f:
    for line in f:
        fragments.append(json.loads(line))
texts = [f["text"] for f in fragments]

# 2) crear modelo y calcular embeddings por batch
model = SentenceTransformer(EMBED_MODEL)
BATCH = 64
embeddings = []
for i in range(0, len(texts), BATCH):
    batch_texts = texts[i:i+BATCH]
    embs = model.encode(batch_texts, show_progress_bar=False, convert_to_numpy=True)
    embeddings.append(embs)
embeddings = np.vstack(embeddings).astype("float32")

# 3) normalizar (si usas coseno)
faiss.normalize_L2(embeddings)

# 4) crear índice FAISS (IndexFlatIP para coseno con embeddings normalizados)
d = embeddings.shape[1]
index = faiss.IndexFlatIP(d)   # inner product = cosine after normalize
index.add(embeddings)
print("FAISS: vectores añadidos:", index.ntotal)

# 5) guardar índice y metadata
faiss.write_index(index, INDEX_FILE)
with open(META_FILE, "w", encoding="utf-8") as f:
    for frag in fragments:
        f.write(json.dumps({
            "id": frag["id"],
            "text": frag["text"],
            "drug": frag.get("drug",""),
            "gene": frag.get("gene",""),
            "evidence": frag.get("evidence",""),
            "source": frag.get("source","")
        }, ensure_ascii=False) + "\n")

print(f"Índice guardado en: {INDEX_FILE}")
print(f"Metadatos guardados en: {META_FILE}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS: vectores añadidos: 26
Índice guardado en: indexes/faiss_index.bin
Metadatos guardados en: indexes/metadata.jsonl
